## REST API Implementation in Python

In [11]:
import flask
import json
import sqlite3

app = flask.Flask(__name__)

database='sportsbook'
conn = sqlite3.connect(database,timeout=10,check_same_thread=False) 
c = conn.cursor()


# Creating database tables and loading data
def create_db_tables(database):

    c.execute('''
              CREATE TABLE IF NOT EXISTS sports
              ([sports_id] INTEGER PRIMARY KEY, [sports_name] TEXT, [s_active] INTEGER,
              UNIQUE(sports_id, sports_name, s_active))
              ''')
          
    c.execute('''
              CREATE TABLE IF NOT EXISTS events
              ([event_id] INTEGER PRIMARY KEY, [sports_id] INTEGER, [event_name] TEXT,
               [e_active] INTEGER, [type] TEXT, [status] TEXT, [scheduled_start] TEXT, [actual_start] TEXT,
               UNIQUE(event_id, sports_id, event_name, e_active, type, status, scheduled_start, actual_start))
              ''')

    c.execute('''
              CREATE TABLE IF NOT EXISTS selections
              ([selection_id] INTEGER PRIMARY KEY, [selection_name] TEXT, [event_id] INTEGER, [event_name] TEXT, 
              [price] REAL, [active] INTEGER, [outcome] TEXT,
              UNIQUE(selection_id, selection_name, event_id, event_name, price, active, outcome))
              ''')
    
    c.execute('''
              INSERT OR IGNORE INTO sports (sports_id,sports_name,s_active)

                    VALUES
                    (1,'football',1),
                    (2,'badminton',1),
                    (3,'basketball',1),
                    (4,'tennis',1),
                    (5,'cricket',1)

              ''')

    c.execute('''
              INSERT OR IGNORE INTO events (event_id,sports_id,event_name,e_active,type,
              status,scheduled_start,actual_start)

                    VALUES
                    (1,1,'European_Cup',1,'preplay','started','2022-21-01 22:38:31','2022-21-01 22:38:31'),
                    (2,1,'FIFA_World_Cup',1,'preplay','pending','2022-22-01 22:38:31','2022-22-01 22:38:31'),
                    (3,3,'World_Cup',0,'inplay','ended','2022-23-01 22:38:31','2022-23-01 22:38:31'),
                    (4,3,'NBA_finals',1,'inplay','pending','2022-24-01 22:38:31','2022-24-01 22:38:31'),
                    (5,5,'IPL',0,'preplay','cancelled','2022-25-01 22:38:31','2022-25-01 22:38:31')

              ''')

    c.execute('''
              INSERT OR IGNORE INTO selections (selection_id,selection_name,event_id,event_name,price,
              active,outcome)

                    VALUES
                    (1,'s1',1,'European_Cup',100.12,1,'win'),
                    (2,'s2',1,'European_Cup',150.00,1,'lose'),
                    (3,'s3',2,'FIFA_World_Cup',200.00,0,'void'),
                    (4,'s4',3,'World_Cup',200.00,1,'unsettled'),
                    (5,'s5',4,'NBA_finals',300.00,1,'void'),
                    (6,'s6',5,'IPL',100.00,0,'void'),
                    (7,'s7',5,'IPL',100.00,0,'void')

              ''')
    
try:
    create_db_tables(database)
    print("Successful : Database & tables created ")
except:
    print("Failed : Database not created ")
    
    
# Function to reflect changes in active state from selections->events->sports
# If all selections of a particular event are inactive, the event becomes inactive
# If all events of a particular sport are inactive, the sport becomes inactive

def update_active():
    c.execute('''
          UPDATE events SET e_active=CASE 
          WHEN event_id IN (SELECT DISTINCT a.event_id FROM events a INNER JOIN selections b ON a.event_id==b.event_id WHERE b.active=1) THEN 1
          ELSE 0
          END
          ''')
    c.execute('''
          UPDATE sports SET s_active=CASE 
          WHEN sports_id IN (SELECT DISTINCT a.sports_id FROM sports a INNER JOIN events b ON a.sports_id==b.sports_id WHERE b.e_active=1) THEN 1
          ELSE 0
          END
          ''')

update_active()


# Home page
@app.route('/', methods=['GET'])
def home():
    return "Sports Book : Web App for managing sports, events and selections"


# Method to GET all sports
@app.route('/sports', methods=['GET'])
def get_sports():
    json_list = []
    conn.row_factory = sqlite3.Row
    c.execute('SELECT * FROM sports;')
    sport=c.fetchall()    
    for row in sport:
        json_dict = {'sports_id': row[0], 'sports_name': row[1], 's_active': row[2]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    json_output = {'sports': x}
    return json_output


# Method to GET all events
@app.route('/events', methods=['GET'])
def get_active_events():
    json_list = []              
    conn.row_factory = sqlite3.Row
    #cur = conn.cursor()
    c.execute('SELECT * FROM events WHERE e_active==1;')
    event=c.fetchall()    
    for row in event:
        json_dict = {'event_name': row[2], 'type ': row[5], 'scheduled_start': row[7]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    json_output = {'events': x}
    return json_output

# Method to GET all sports,events & selections
@app.route('/all', methods=['GET'])
def get_all():
    json_list = []              
    conn.row_factory = sqlite3.Row
    c.execute('''
              SELECT a.sports_name,b.event_name,b.type,b.status,b.scheduled_start,c.selection_name,c.price,c.outcome
              FROM sports a
              INNER JOIN events b ON a.sports_id = b.sports_id
              INNER JOIN selections c ON c.event_id=b.event_id
              ''')
    event=c.fetchall()    
    for row in event:
        json_dict = {'sports_name': row[0], 'event_name ': row[1], 'type': row[2], 'status': row[3], 'scheduled_start': row[4],
                    'selection_name': row[0],'price': row[0],'outcome': row[0]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    json_output = {'events': x}
    return json_output


# Method to get only winning selections
@app.route('/win', methods=['GET'])
def winning_selections():
    json_list = []              
    conn.row_factory = sqlite3.Row
    #cur = conn.cursor()
    c.execute('''
              SELECT a.event_name,a.type,a.actual_start,b.selection_name,b.price,b.outcome
              FROM events a
              INNER JOIN selections b ON a.event_id=b.event_id
              WHERE outcome=='win'
              ''')
    event=c.fetchall()    
    for row in event:
        json_dict = {'event_name': row[0], 'type ': row[1], 'actual_start': row[2], 'selection_name': row[3], 
                     'price': row[4],'outcome': row[5]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    json_output = {'events': x}
    return json_output


# Method to create/POST sports
@app.route('/sports/add',  methods=['POST'])
def insert_sport():
    inserted_sport = {}
    try:
        c.execute('''INSERT INTO sports (sports_id, sports_name, s_active) 
                    VALUES (?, ?, ?, ?, ?)''', (sports['sports_id'],   
                    sports['sports_name'], sports['s_active']) )
        conn.commit()
        inserted_sport = c.lastrowid
    except:
        conn().rollback()

    update_active()
    return inserted_sport


# Method to Update/PUT events
@app.route('/events/update',  methods=['PUT'])
def update_events():
    c.execute('''UPADTE events SET event_name='Euro_Cup' WHERE event_name='European_Cup' ''')
    conn.commit()
    update_active()
    return
        

#Main function
if __name__ == '__main__':
    app.run()
    update_active()

Successful : Database & tables created 
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jan/2022 08:51:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2022 08:52:02] "GET /sports HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2022 08:52:10] "GET /events HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2022 08:52:16] "GET /all HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2022 08:52:52] "GET /win HTTP/1.1" 200 -
